# Aicon 1

## Action Making

In [1]:
# Chunk 1: Imports
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# Import the required classes
from aicons.definitions.aicon import AIcon
from aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor import MetaAdsSalesSensor

In [2]:
# Chunk 2: Create AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: marketing_aicon


In [3]:
aicon.add_state_factor(
    name="purchases",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 1.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

aicon.add_state_factor(
    name="add_to_carts",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 5.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

aicon.add_state_factor(
    name="initiated_checkouts",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 2.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

ContinuousLatentVariable(name='initiated_checkouts', value=0.0, description='')

In [4]:
# Chunk 3: Setup Meta Ads sensor
access_token = "EAAZAn8wmq1IEBOZCz8oyDZBBgiazAgnQKIoAr4mFTbkV7jxi6t3APzOSxFybXNIkBgwQACdagbs5lFE8tpnNOBOOpWtS3KjZAdf9MNAlySpwEaDrX32oQwUTNmOZAaSXjT5Os5Q8YqRo57tXOUukB7QtcO8nQ8JuqrnnshCr7A0giynZBnJKfuPakrZBWoZD"
ad_account_id = "act_252267674525035"
campaign_id = "120218631288730217"

sensor = MetaAdsSalesSensor(
    name="meta_ads",
    reliability=0.9,
    access_token=access_token,
    ad_account_id=ad_account_id,
    campaign_id=campaign_id,
    api_version="v18.0",
    time_granularity="hour"
)

aicon.add_sensor("meta_ads", sensor)
print("Meta Ads sensor added")

✅ Successfully initialized Meta Ads API vv18.0 for account act_252267674525035
Found 5 ad sets in campaign 120218631288730217
Found 19 ads in 5 ad sets
Found 3 active ads out of 19 total ads
Found 3 active ads
- Ad ID: 120219835173950217, Name: Anuncio 9 - Nata + Gabito
- Ad ID: 120218907658700217, Name: Anuncio 4 - Carrusel 1 - Rancho
- Ad ID: 120218905466570217, Name: Anuncio 2 - Carrusel 1 (3 artistas)
Number of factors: 21
Factor names: purchases, add_to_carts, initiated_checkouts, ad_120219835173950217_purchases, ad_120219835173950217_add_to_carts, ad_120219835173950217_initiated_checkouts, ad_120219835173950217_impressions, ad_120219835173950217_clicks, ad_120219835173950217_spend, ad_120218907658700217_purchases...
Auto-creating missing factor: ad_120219835173950217_purchases (continuous)
Auto-creating missing factor: ad_120219835173950217_add_to_carts (continuous)
Auto-creating missing factor: ad_120219835173950217_initiated_checkouts (continuous)
Auto-creating missing factor: 

In [5]:
# Chunk 4: Define action space
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

ActionSpace(dimensions=2, constraints=1)

In [6]:
print(aicon.brain.action_space.raw_print())

dimensions: ['google_budget: [0.0, 1000.0] step=100.0', 'facebook_budget: [0.0, 1000.0] step=100.0']
size: 121 (stepped continuous)
constraints: ['<function create_marketing_ads_space.<locals>.budget_sum_constraint at 0x1618ed6c0>']
actions: [{'google_budget': np.float64(0.0), 'facebook_budget': np.float64(1000.0)}, {'google_budget': np.float64(100.0), 'facebook_budget': np.float64(900.0)}, {'google_budget': np.float64(200.0), 'facebook_budget': np.float64(800.0)}, {'google_budget': np.float64(300.0), 'facebook_budget': np.float64(700.0)}, {'google_budget': np.float64(400.0), 'facebook_budget': np.float64(600.0)}, {'google_budget': np.float64(500.0), 'facebook_budget': np.float64(500.0)}, {'google_budget': np.float64(600.0), 'facebook_budget': np.float64(400.0)}, {'google_budget': np.float64(700.0), 'facebook_budget': np.float64(300.0)}, {'google_budget': np.float64(800.0), 'facebook_budget': np.float64(200.0)}, {'google_budget': np.float64(900.0), 'facebook_budget': np.float64(100.0)}

In [7]:
# Chunk 5: Define utility function
utility = aicon.define_utility_function(
    utility_type='linear',
    name="Marketing Budget Utility",
    weights={'google': 0.6, 'facebook': 0.4}
)


Utility Function: LinearUtility: Σ(weights * values) where weights = [tf.Tensor(0.6, shape=(), dtype=float32), tf.Tensor(0.4, shape=(), dtype=float32)]


In [8]:
# Chunk 6: Update from Meta Ads sensor
print("Updating beliefs from Meta Ads sensor...")
aicon.update_from_sensor("meta_ads")

Updating beliefs from Meta Ads sensor...

=== Starting Belief Update ===
Current time: 2025-03-29 14:21:29
No posterior samples available
Campaign objective: OUTCOME_SALES
Found 5 ad sets in campaign 120218631288730217
Found 19 ads in 5 ad sets
Trying date_preset: today
Fetching data for 19 ads with today preset...
Retrieved 52 records
Found latest hour: 14:00:00 - 14:59:59
Filtered to 3 records for hour 14:00:00 - 14:59:59
Found 3 hourly records
Extracted factors for 3 ads
Example ad IDs: 120218905466570217, 120218907658700217, 120219835173950217
Example factor names: purchases, add_to_carts, initiated_checkouts, ad_120218905466570217_purchases, ad_120218905466570217_add_to_carts, ad_120218905466570217_initiated_checkouts, ad_120218905466570217_impressions, ad_120218905466570217_clicks, ad_120218905466570217_spend, ad_120218907658700217_purchases
Mapping observation: purchases → purchases
Mapping observation: add_to_carts → add_to_carts
Mapping observation: initiated_checkouts → initi

/Users/infa/Library/Caches/pypoetry/virtualenvs/babel-PAVoc7qP-py3.12/lib/python3.12/site-packages/tensorflow_probability/python/__init__.py:70: UserWarning: TensorFloat-32 matmul/conv are enabled for NVIDIA Ampere+ GPUs. The resulting loss of precision may hinder MCMC convergence. To turn off, run `tf.config.experimental.enable_tensor_float_32_execution(False)`. For more detail, see https://github.com/tensorflow/community/pull/287.
  warnings.warn(


Prior log probability: Tensor("mcmc_sample_chain/simple_step_size_adaptation___init__/_bootstrap_results/mh_bootstrap_results/hmc_kernel_bootstrap_results/maybe_call_fn_and_grads/value_and_gradients/value_and_gradient/JointDistributionSequential_CONSTRUCTED_AT_top_level/log_prob/add_20:0", shape=(), dtype=float32)

Processing factor: purchases
Observation value: 0
Observation type: <class 'tensorflow.python.framework.ops.EagerTensor'>
  Observation tensor dtype: <dtype: 'int32'>
  Observation tensor shape: ()
Sampled value: Tensor("mcmc_sample_chain/current_state:0", shape=(), dtype=float32)
Sampled value type: <class 'tensorflow.python.framework.ops.SymbolicTensor'>
  Sampled value tensor dtype: <dtype: 'float32'>
  Sampled value tensor shape: ()
Reliability: 0.9
Factor type: continuous
Factor params: {'loc': 0.0, 'scale': 1.0}
Continuous factor log likelihood: Tensor("mcmc_sample_chain/simple_step_size_adaptation___init__/_bootstrap_results/mh_bootstrap_results/hmc_kernel_bootstrap_r

True

In [ ]:
# Get best action using the posterior samples from the update
best_action, expected_utility = aicon.brain.find_best_action()
print(f"\nBest action: {best_action}")
print(f"Expected utility: {expected_utility:.4f}")


-------------------

In [1]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# Import all necessary modules first
import aicons.bayesbrainGPT.bayes_brain_ref
import aicons.bayesbrainGPT.utility_function.utility_base
import aicons.bayesbrainGPT.utility_function.linear_utility
import aicons.bayesbrainGPT.utility_function
import aicons.definitions.aicon

# Now we can reload them
importlib.reload(aicons.bayesbrainGPT.bayes_brain_ref)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_base)
importlib.reload(aicons.bayesbrainGPT.utility_function)  # Reload the package first
importlib.reload(aicons.definitions.aicon)
importlib.reload(aicons.bayesbrainGPT)
importlib.reload(aicons)

# Import the classes we need
from aicons.definitions.aicon import AIcon

In [ ]:
# Create the AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

### Defining Action Space

In [ ]:
# Define an action space
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

In [ ]:
print(aicon.brain.action_space.raw_print()) 

### Defining Utility Function

In [ ]:
utility = aicon.define_utility_function(
    utility_type='linear',
    name="Marketing Budget Utility",
    weights={'google': 0.6, 'facebook': 0.4}
)

### Defining State Rep State

In [ ]:
# Add root factors (with empty relationships)
aicon.add_state_factor(
    name="market_size",
    factor_type="continuous",
    value=10000.0,
    params={
        "loc": 10000.0,
        "scale": 1000.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

aicon.add_state_factor(
    name="competition_level",
    factor_type="categorical",
    value="medium",
    params={
        "categories": ["low", "medium", "high"],
        "probs": [0.2, 0.5, 0.3]
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

# Add dependent factors
aicon.add_state_factor(
    name="conversion_rate",
    factor_type="continuous",
    value=0.02,
    params={
        "loc": 0.02,
        "scale": 0.005,
        "constraints": {"lower": 0, "upper": 1}
    },
    relationships={
        "depends_on": ["market_size", "competition_level"]
    }
)

aicon.add_state_factor(
    name="customer_acquisition_cost",
    factor_type="continuous",
    value=50.0,
    params={
        "loc": 50.0,
        "scale": 10.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": ["competition_level"]
    }
)

# Add a discrete factor for number of competitors
aicon.add_state_factor(
    name="num_competitors",
    factor_type="discrete",
    value=5,
    params={
        "rate": 5.0  # Poisson distribution parameter
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

In [ ]:
print(aicon.brain.state)

### Perception

In [ ]:
# Try to update beliefs without any sensors
print("Current state before update:")
print(aicon.get_state())

## Sensors

In [1]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# Import all necessary modules first
import aicons.bayesbrainGPT.bayes_brain_ref
import aicons.bayesbrainGPT.utility_function.utility_base
import aicons.bayesbrainGPT.utility_function.linear_utility
import aicons.bayesbrainGPT.utility_function
import aicons.definitions.aicon

# Now we can reload them
importlib.reload(aicons.bayesbrainGPT.bayes_brain_ref)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_base)
importlib.reload(aicons.bayesbrainGPT.utility_function)  # Reload the package first
importlib.reload(aicons.definitions.aicon)
importlib.reload(aicons.bayesbrainGPT)
importlib.reload(aicons)

# Import the classes we need
from aicons.definitions.aicon import AIcon
from aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor import MetaAdsSalesSensor


In [ ]:
# Create the AIcon
aicon = AIcon("marketing_aicon")
print(f"AIcon created: {aicon.name}")

In [ ]:
# Define an action space
aicon.define_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=2,
    budget_step=100.0,
    ad_names=['google', 'facebook']
)

In [ ]:
print(aicon.brain.action_space.raw_print())

In [ ]:
# Define a utility function
utility = aicon.define_utility_function(
    utility_type='linear',
    name="Marketing Budget Utility",
    weights={'google': 0.6, 'facebook': 0.4}
)

In [ ]:
aicon.add_state_factor(
    name="purchases",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 1.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

aicon.add_state_factor(
    name="add_to_carts",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 5.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

aicon.add_state_factor(
    name="initiated_checkouts",
    factor_type="continuous",
    value=0.0,
    params={
        "loc": 0.0,
        "scale": 2.0,
        "constraints": {"lower": 0}
    },
    relationships={
        "depends_on": []  # Empty list for root factor
    }
)

In [ ]:
print(aicon.brain.state)

In [ ]:
# Your Meta API credentials
access_token = "EAAZAn8wmq1IEBOZCz8oyDZBBgiazAgnQKIoAr4mFTbkV7jxi6t3APzOSxFybXNIkBgwQACdagbs5lFE8tpnNOBOOpWtS3KjZAdf9MNAlySpwEaDrX32oQwUTNmOZAaSXjT5Os5Q8YqRo57tXOUukB7QtcO8nQ8JuqrnnshCr7A0giynZBnJKfuPakrZBWoZD"
ad_account_id = "act_252267674525035"
campaign_id = "120218631288730217"

# Create the Meta Ads Sensor
sensor = MetaAdsSalesSensor(
    name="meta_ads",
    reliability=0.9,
    access_token=access_token,
    ad_account_id=ad_account_id,
    campaign_id=campaign_id,
    api_version="v18.0",
    time_granularity="hour"
)

# Add the sensor to the AIcon
aicon.add_sensor("meta_ads", sensor)
print("Meta Ads sensor added")

In [ ]:
# Get active ads to understand what we're working with
active_ads = sensor.get_active_ads()
print(f"Found {len(active_ads)} active ads")
for ad in active_ads:
    print(f"- Ad ID: {ad['ad_id']}, Name: {ad['ad_name']}")

In [ ]:
aicon.update_from_sensor("meta_ads")